<a href="https://colab.research.google.com/github/arpitnigam45/ResumeAnalysis/blob/main/NLP_Resume_Screening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Pre-processing

In [ ]:
!pip install PyPDF2
!pip install python-docx
!pip install textract
!apt-get install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr \
flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig
!pip install -U nltk
!pip install wget
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!pip install matplotlib
!pip install pandas
!pip install wordcloud
!pip install pdfminer3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
python-dev is already the newest version (2.7.15~rc1-1).
antiword is already the newest version (0.37-11build1).
flac is already the newest version (1.3.2-1).
lame is already the newest version (3.100-2).
pstotext is already the newest version (1.9-6build1).
swig is already the newest version (3.0.12-1).
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
unrtf is already the newest version (0.21.9-clean-3).
libxml2-dev is already the newest version (2.9.4+dfsg1-6.1ubuntu1.3).
libxslt1-dev is already the newest version (1.1.29-5ubuntu0.2).
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
libmad0 is already the newest version (0.15.1b-9ubuntu18.04.1).
libsox-fmt-mp3 is already the newest version (14.4.2-3ubuntu0.18.04.1).
sox is already the ne

In [ ]:
import os
import textract
import docx
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
import nltk
import pandas as pd
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
resumeDir = '/content/drive/My Drive/data/raw_resume/'
sk=pd.DataFrame()


In [ ]:
def extract_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = r.findall(string)
    return [re.sub(r'\D', '', number) for number in phone_numbers if len(number)==10]

In [ ]:
def extract_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)

In [ ]:
def convert_pdf_to_txt(path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(path, 'rb') as fh:

        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    # close open handles
    converter.close()
    fake_file_handle.close()

    return text


In [ ]:
import re
resumeTxt = []
for filename in os.listdir(resumeDir):
    if(filename.endswith(".pdf")):
        try:
            resumeTxt.append(convert_pdf_to_txt(resumeDir+filename))
        except Exception:
            print('Error reading .pdf file' + filename)
    if(filename.endswith(".docx") and bool(re.search(r"\s", filename)==False)):
        try:
            doc = docx.Document(resumeDir + filename)
            data = ""
            fullText = []
            for para in doc.paragraphs:
                fullText.append(para.text)
                data = '\n'.join(fullText)
            resumeTxt.append(data)
        except IOError:
            print('Error reading .docx file' + filename)
    if(filename.endswith(".doc")):
        continue

In [ ]:
  len(resumeTxt) 

1035

In [ ]:
print(resumeTxt[0])

Richard Chua, CPA, CMA 
Vice President, Compliance Consulting and Internal Audit 
Contact Numbers: +65 8693 9228 
E-mail address: richardachua@yahoo.com 
 
 
 
Executive Summary  
 
Richard is a Certified Public Accountant and a Certified Management Accountant with a cumulative 
experience of almost 13 years in various finance, compliance and management role, almost 9 years 
of  which  are  from  external  audit  with  Ernst  &  Young.    Richard’s  overall  experience  ranges  from 
financial audit, financial analysis and reporting, internal controls reporting, compliance requirements 
and  operations  management.    His  main  expertise  is  in  the  asset  management  operation,  financial 
reporting  and  compliance.    He  has  notable  exposure  in  financial  reporting  assisting  clients  with 
complex  financial  reporting  issues  such  as  consolidation,  investment  valuation,  reviewing  financial 
statements  and  corresponding  disclosures.  He  is  proficient  in  IFRS,

In [ ]:
email=list()
phone=list()
for resume in resumeTxt:
    tok_resume = ''
    for tok in resume:
        tok_resume = tok_resume + tok + ' '
    email.append(extract_email_addresses(resume))
    phone.append(extract_phone_numbers(resume))

In [ ]:
sk.insert(0,"Phone",phone,True)
sk.insert(0,"Email",email,True)

In [ ]:
from nltk.corpus import stopwords
sw = set(stopwords.words('english'))

In [ ]:
resumeTxtNoStopword = []

for resume in resumeTxt:
    text = resume
    text = text.split()
    useful_words = [w for w in text if w not in sw]
    resumeTxtNoStopword.append(" ".join(useful_words))

In [ ]:
len(resumeTxtNoStopword)

1035

In [ ]:
import re
def tokenize(text):
    # obtains tokens with a least 1 alphabet
    pattern = re.compile(r'[A-Za-z]+-?[\w^\']*|[\w^\']*[A-Za-z]+[\w^\']*')
    return pattern.findall(text.lower())

In [ ]:
i = 0
tokenized_resumes = []
for resume in resumeTxtNoStopword:
    r = tokenize(resume)
    tokenized_resumes.append(r)
    i += 1

In [ ]:
print(tokenized_resumes[:3])

[['richard', 'chua', 'cpa', 'cma', 'vice', 'president', 'compliance', 'consulting', 'internal', 'audit', 'contact', 'numbers', 'e-mail', 'address', 'richardachua', 'yahoo', 'com', 'executive', 'summary', 'richard', 'certified', 'public', 'accountant', 'certified', 'management', 'accountant', 'cumulative', 'experience', 'almost', 'years', 'various', 'finance', 'compliance', 'management', 'role', 'almost', 'years', 'external', 'audit', 'ernst', 'young', 'richard', 's', 'overall', 'experience', 'ranges', 'financial', 'audit', 'financial', 'analysis', 'reporting', 'internal', 'controls', 'reporting', 'compliance', 'requirements', 'operations', 'management', 'his', 'main', 'expertise', 'asset', 'management', 'operation', 'financial', 'reporting', 'compliance', 'he', 'notable', 'exposure', 'financial', 'reporting', 'assisting', 'clients', 'complex', 'financial', 'reporting', 'issues', 'consolidation', 'investment', 'valuation', 'reviewing', 'financial', 'statements', 'corresponding', 'disclo

In [ ]:
txt_file = '/content/drive/My Drive/data/clean_resume/resume'

In [ ]:
i = 0
for resume in tokenized_resumes:
    tok_resume = ''
    for tok in resume:
        tok_resume = tok_resume + tok + ' '
    file = open(txt_file + str(i), 'w+')
    file.writelines(tok_resume)
    file.close()
    i += 1

#Model Building


In [ ]:
from gensim.models.phrases import Phrases, Phraser
from collections import defaultdict
import multiprocessing
from gensim.models import Word2Vec
import time

In [ ]:
#self
sent = [row for row in tokenized_resumes]
phrases = Phrases(sent, min_count=30, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[sent]

In [ ]:
#extraction
w2v_model =Word2Vec(sentences, size=300, window=12, min_count=5, negative=15,iter=15, workers=multiprocessing.cpu_count())
w2v_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz', lockf=1.0, binary=True)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs = 10)

(4162231, 4586050)

In [ ]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

26248

In [ ]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['management',
 'singapore',
 'financial',
 'business',
 'audit',
 'clients',
 'investment',
 's',
 'company',
 'team']

In [ ]:
w2v_model.wv.most_similar(positive=["bank"])

[('vickers', 0.5045034289360046),
 ('bny', 0.48627832531929016),
 ('branch', 0.4622983932495117),
 ('banking_corporation', 0.46201878786087036),
 ('relationship_manager', 0.44938525557518005),
 ('banking', 0.43962377309799194),
 ('abn', 0.4325631260871887),
 ('amro', 0.43202441930770874),
 ('banks', 0.4148557484149933),
 ('boc', 0.41345784068107605)]

#Skill Score

In [ ]:
skills=['bank','trade','teach','engineer','driver','writer','data','cloud','programming','administration','sales','invest']
job_skills=dict()
for i in skills:
  l=w2v_model.wv.most_similar(positive=i)
  p=[]
  for j in l:
    p.append(j[0])
  job_skills.__setitem__(i,p)
  

In [ ]:
import pandas as pd
def compute_score(i,sk,x):  
  l=list()
  for j in range(0,len(tokenized_resumes)):
      c=0
      file=open(txt_file + str(j) ,'r')
      for line in file:
          # reading each word        
          for word in line.split():
              # displaying the words
              pr_sc=dict()
              if word in job_skills[i]:
                  c=c+1
      l.append(c)
  sk.insert(x, i, l, True)

In [ ]:
x=0
for i in skills:
  compute_score(i,sk,x)
  x=x+1

In [ ]:
sk['bank']=sk['bank']+sk['invest']
sk=sk.drop(['invest'],axis=1)

In [ ]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns[0:-2]:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result
sk=normalize(sk)
print(sk)

          bank  ...         Phone
0     0.191489  ...            []
1     0.021277  ...            []
2     0.000000  ...  [1803818953]
3     0.021277  ...            []
4     0.063830  ...            []
...        ...  ...           ...
1030  0.021277  ...            []
1031  0.127660  ...            []
1032  0.000000  ...            []
1033  0.042553  ...            []
1034  0.021277  ...            []

[1035 rows x 13 columns]


In [ ]:
l=[]
for i in range(0,len(tokenized_resumes)):
  s='resume'+str(i)
  l.append(s)
sk.insert(0,"Resume_Id",l,True)

In [ ]:
a=input("The parameter wanted:")
(sk.sort_values(by=a, ascending=False)).head()

The parameter wanted:bank


,Resume_Id,bank,trade,teach,engineer,driver,writer,data,cloud,programming,administration,sales,Email,Phone
1021,resume1021,1.000000,0.071429,0.142857,0.0,0.133333,0.0,0.020134,0.000000,0.0,0.0,0.500000,[pnarula71@rediffmail.com],[9810029640]
481,resume481,0.319149,0.000000,0.000000,0.0,0.066667,0.0,0.006711,0.000000,0.0,0.0,0.318182,[perline_tan@yahoo.com],[]
302,resume302,0.319149,0.178571,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.454545,[hamed@westviewcapitaladvisors.com],[9715520855]
980,resume980,0.297872,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.045455,[Rebecca.Dengxw@gmail.com],[]
685,resume685,0.276596,0.071429,0.000000,0.3,0.066667,0.0,0.013423,0.266667,0.0,0.0,0.181818,[ray.ls.yeung@gmail.com],[]


In [ ]:
a=input("The parameter wanted:")
(sk.sort_values(by=a, ascending=False)).head()

The parameter wanted:teach


,Resume_Id,bank,trade,teach,engineer,driver,writer,data,cloud,programming,administration,sales,Email,Phone
815,resume815,0.085106,0.000000,1.000000,0.0,0.066667,0.0,0.000000,0.0,0.000000,0.060606,0.0,[luong_quangninh@yahoo.com],[]
394,resume394,0.000000,0.000000,0.857143,0.0,0.000000,0.0,0.006711,0.0,0.000000,0.000000,0.0,[shawnwwsean@gmail.com],[]
413,resume413,0.000000,0.035714,0.857143,0.0,0.000000,0.0,0.013423,0.0,0.038462,0.000000,0.0,[runchao91@gmail.com],[]
164,resume164,0.021277,0.035714,0.571429,0.0,0.000000,0.0,0.013423,0.0,0.000000,0.060606,0.0,[hyyalina@gmail.com],[]
371,resume371,0.000000,0.000000,0.571429,0.0,0.000000,0.0,0.006711,0.0,0.000000,0.030303,0.0,[kevinauck@gmail.com],[]
